Imports

In [50]:
from PIL import Image
import numpy as np
import cv2
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [51]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

- Binary to Decimal Converter

In [52]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [53]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


## 2D Chaotic Map

- Hennon Map (Binary)

In [54]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

## Encoding


In [58]:
def encode(videoname, filename, hennonKey, poly):

    # Divide video into frames
    vidObj = cv2.VideoCapture(videoname) 
    success = True
    frames = []
    while success: 
        success, image = vidObj.read()
        frames.append(image)

    frame_usage = [0 for i in range(len(frames))]
    
    # Set Limit
    frame_size = (len(frames[0]), len(frames[0][0]))
    print(frame_size)
    frame_usage_limit = (frame_size[0] * frame_size[1] * 3) * 0.3

    # Get secret data
    filehandler = open(filename,"r")
    secret_data = "".join(filehandler.readlines())

    # convert data to binary
    binary_secret_data = convertToBinary(secret_data)
    data_len = len(binary_secret_data)
    print(data_len)

    data_index = 0

    # Build LFSR to be used to choose frame
    frame_lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)
    frame_index = 0

    while data_index < data_len:

        print("frame loop entered")

        # Choose Frame using LFSR
        frame_index = (frame_index + binaryToDecimal(frame_lfsr.state)) % len(frames)
        frame_lfsr.next()

        # If frame is already used, keep choosing next frame until unused frame is found.
        count = 0
        while frame_usage[frame_index] != 0:
            frame_index = (frame_index + binaryToDecimal(frame_lfsr.state)) % len(frames)
            frame_lfsr.next()
            count += 1
            if count > 100:
                print("frames exceeded")
                return
            
        print("frame selected", frame_index)

        # Build chaotic map and derive some available pixels for the frame
        map = HennonMapBinary(frame_size, hennonKey)

        print("checkpoint 1")

        availablePixels = []

        for i in range(frame_size[0]):
            for j in range(0,frame_size[1]):
                if map[i][j] == 1:
                    availablePixels.append([i,j,0])
        
        print("checkpoint 2")

        # Build LFSR for pixel choice
        pixel_lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)
        pixel_index = 0

        print("map created")

        while data_index < data_len:

            print("pixel loop entered", data_index)

            # Choose pixel using LFSR from Available pixels
            pixel_index = (pixel_index + binaryToDecimal(pixel_lfsr.state)) % len(availablePixels)
            pixel_lfsr.next()

            # If pixel is already used 3 times, keep choosing next pixel until unused pixel is found.
            count = 0
            while availablePixels[pixel_index][2] > 2:
                pixel_index = (pixel_index + binaryToDecimal(pixel_lfsr.state)) % len(availablePixels)
                pixel_lfsr.next()
                count += 1
                if count > 100:
                    print("pixels exceeded")
                    return
            
            print("pixel selected", pixel_index)

            '''
            EACH PIXEL CAN BE USED AT MAX 3 TIMES. WE KEEP COUNT OF NUMBER OF USES AS USAGESTATE.

            IF USAGESTATE IS 0: XOR 2 DATA BITS WITH 2 LSB OF RED VALUE
            IF USAGESTATE IS 1: XOR 2 DATA BITS WITH 2 LSB OF BLUE VALUE
            IF USAGESTATE IS 2: XOR 2 DATA BITS WITH 2 LSB OF GREEN VALUE
            '''

            # Extract row, column and usagestate values.
            row = availablePixels[pixel_index][0]
            col = availablePixels[pixel_index][1]
            usageState = availablePixels[pixel_index][2]

            # Update with Red Value
            if usageState == 0:
                red = list(convertToBinary(frames[frame_index][row,col][0]))
                red[-2] = str(int(red[-2]) ^ int(binary_secret_data[data_index]))
                red[-1] = str(int(red[-1]) ^ int(binary_secret_data[data_index+1]))
                red = "".join(red)
                red = binaryToDecimal(red)
                frames[frame_index][row,col] = (red, frames[frame_index][row,col][1], frames[frame_index][row,col][2])

            # Update with Blue Value
            elif usageState == 1:
                blue = list(convertToBinary(frames[frame_index][row,col][1]))
                blue[-2] = str(int(blue[-2]) ^ int(binary_secret_data[data_index]))
                blue[-1] = str(int(blue[-1]) ^ int(binary_secret_data[data_index+1]))
                blue = "".join(blue)
                blue = binaryToDecimal(blue)
                frames[frame_index][row,col] = (frames[frame_index][row,col][0], blue, frames[frame_index][row,col][2])
            
            # Update with Green Value
            elif usageState == 2:
                green = list(convertToBinary(frames[frame_index][row,col][2]))
                green[-2] = str(int(green[-2]) ^ int(binary_secret_data[data_index]))
                green[-1] = str(int(green[-1]) ^ int(binary_secret_data[data_index+1]))
                green = "".join(green)
                green = binaryToDecimal(green)
                frames[frame_index][row,col] = (frames[frame_index][row,col][0], frames[frame_index][row,col][1], green)
            
            print(data_index, pixel_index)
            data_index += 2
            availablePixels[pixel_index][2] += 1
            frame_usage[frame_index] += 1
            if frame_usage[frame_index] >= frame_usage_limit:
                break

    video = cv2.VideoWriter("images/encoded_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_size[1], frame_size[0]))
    for frame in frames:
        video.write(frame)
    video.release()

In [59]:
import json

def main():
    info = open("config.json","r")
    json_data = json.load(info)
    original_video = json_data["source"]["video_filename"]
    hennon_key = json_data["key"]["hennon_key"]
    lfsr_poly = json_data["key"]["lfsr_polynomial"]
    
    encode(original_video, "text\input.txt", hennon_key, lfsr_poly)

In [60]:
main()

(720, 1280)
64
frame loop entered
frame selected 103
checkpoint 1
checkpoint 2
map created
pixel loop entered 0
pixel selected 255
0 255
pixel loop entered 2
pixel selected 510
2 510
pixel loop entered 4
pixel selected 765
4 765
pixel loop entered 6
pixel selected 1020
6 1020
pixel loop entered 8
pixel selected 1275
8 1275
pixel loop entered 10
pixel selected 1530
10 1530
pixel loop entered 12
pixel selected 1785
12 1785
pixel loop entered 14
pixel selected 2040
14 2040
pixel loop entered 16
pixel selected 2295
16 2295
pixel loop entered 18
pixel selected 2550
18 2550
pixel loop entered 20
pixel selected 2805
20 2805
pixel loop entered 22
pixel selected 3060
22 3060
pixel loop entered 24
pixel selected 3315
24 3315
pixel loop entered 26
pixel selected 3570
26 3570
pixel loop entered 28
pixel selected 3825
28 3825
pixel loop entered 30
pixel selected 4080
30 4080
pixel loop entered 32
pixel selected 4335
32 4335
pixel loop entered 34
pixel selected 4590
34 4590
pixel loop entered 36
pix